In [ ]:
#default_exp core

# rfpfolio

> Create portfolios with rebalancing, and measure performance.

In [ ]:
#hide

# Do this to see possible %nbdev_ magics
from nbdev import *

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

## Load Price Data

In [ ]:
%nbdev_export
import pandas as pd
import numpy as np
import array
import os.path

In [ ]:
%nbdev_export
class PriceSource:
    
    def __init__(self, data_root):
        """
        Args:
            data_root (str): Relative or absolute path to the root directory to load data from.
            In the root should be one or more directories named 'weekly', 'monthly', etc.
        """
        self.data_root = os.path.abspath(data_root)
        
    def __repr__(self):
        return (self.__class__.__qualname__ + f"(data_root={self.data_root})")
    
    def loadAdjustedPrices(self, ticker, subdir='weekly'):
        """
        Load data in to single column dataframe, indexed by date.
        The column has the name of the ticker, and is the adjusted price.
        Note: for Yahoo weekly data, the adj price is the adjusted closing price for the 
        week beginning on the specified date.
        
        Args:
            ticker (str): Name of file to load, without '.csv' extension.
            subdir (str): Subdir of data_root wherein to find file to load.
            
        Returns:
            A dataframe with the specified data, indexed by date.
        """
        csv_path =  os.path.join(self.data_root, subdir, f"{ticker}.csv")
        prices = pd.read_csv(csv_path, parse_dates=[0])
        prices = prices[['Date', 'Adj Close']]
        prices.columns = ['Date', ticker]
        return prices.set_index('Date')
    
    def loadAllAdjustedPrices(self, tik_list, subdir='weekly'):
        """
        Load adjusted price data for all tickers to a dataframe, indexed by date.
        Each column name is a ticker, and the column values are the sequence of adjusted prices.
        
        Returns:
            A dataframe with the specified data, indexed by date.
        """
        tik_dict = {tik:  self.loadAdjustedPrices(tik, subdir) for tik in tik_list}
        return pd.concat([tik_dict[tik] for tik in tik_list], axis=1, join='inner')

Here is an example of loading some weekly data:

In [ ]:
tst_src = PriceSource('testdata/2017-Apr')

In [ ]:
price_data = tst_src.loadAllAdjustedPrices(['SPY', 'IEI', 'GLD'])
price_data.head()

,SPY,IEI,GLD
Date,,,
2017-04-03,220.896225,115.845558,119.459999
2017-04-10,218.369843,116.867851,122.599998
2017-04-17,220.323349,116.943314,122.309998
2017-04-24,223.601120,116.688652,120.769997
2017-05-01,225.122589,116.301956,117.010002


In [ ]:
%nbdev_hide
# top 5 rows of data
spy_5 = price_data.loc[:'2017-05-01', 'SPY']
test_close(list(spy_5), [220.896225, 218.369843, 220.323349, 223.60112, 225.122589])

iei_5 = price_data.loc[:'2017-05-01', 'IEI']
test_close(list(iei_5), [115.845558, 116.867851, 116.943314, 116.688652, 116.301956])

gld_5 = price_data.loc[:'2017-05-01', 'GLD']
test_close(list(gld_5), [119.459999, 122.599998, 122.309998, 120.769997, 117.010002])

# last row of data
last = price_data.iloc[-1:,:]
# 2020-06-29 data for spy, iei, gld
last.to_numpy()[0] = np.array([310.519989, 133.316208, 166.619995])